## Calculate the averge loss and standard deviation of the loss

In [1]:
import os
import re
from collections import defaultdict
from statistics import mean, stdev

def parse_final_losses_from_file(filepath):
    with open(filepath, 'r') as file:
        content = file.read()

    # Split content by blocks between delimiter lines
    blocks = content.split("=" * 80)

    model_losses = defaultdict(list)

    for block in blocks:
        model_match = re.search(r'([^\s]+)\s*->\s*([^\n]+)', block)
        loss_match = re.search(r'Final Loss:\s*([0-9.]+)', block)

        if model_match and loss_match:
            teacher_model = model_match.group(1).strip()
            student_model = model_match.group(2).strip()
            loss = float(loss_match.group(1))

            # Aggregate by teacher -> student combination
            pair_key = f"{teacher_model} -> {student_model}"
            model_losses[pair_key].append(loss)
    return model_losses

def aggregate_losses(folder_path):
    aggregated_losses = defaultdict(list)

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                file_losses = parse_final_losses_from_file(file_path)
                for model, losses in file_losses.items():
                    aggregated_losses[model].extend(losses)
                    
    summary = {}
    for model_pair, losses in aggregated_losses.items():
        avg = mean(losses)
        std = stdev(losses) if len(losses) > 1 else 0.0
        summary[model_pair] = (avg, std)

    return summary


In [12]:
folder = "../experiment_output/Experiment2/exp2_noReg_init0.2"
results = aggregate_losses(folder)

print("Average Final Loss ± Std per Teacher -> Student Model:")
for model_pair, (avg_loss, std_loss) in sorted(results.items()):
    print(f"{model_pair}: {avg_loss:.4f} ± {std_loss:.4f}")

Average Final Loss ± Std per Teacher -> Student Model:
baselineCNN_relu -> multiChannelCNN_relu: 0.0000 ± 0.0000
baselineCNN_relu -> multiChannelCNN_sigmoid: 3.3447 ± 0.0848
baselineCNN_relu -> multiChannelCNN_tanh: 3.6629 ± 0.0827
baselineCNN_sigmoid -> multiChannelCNN_relu: 0.0118 ± 0.0004
baselineCNN_sigmoid -> multiChannelCNN_sigmoid: 0.0000 ± 0.0000
baselineCNN_sigmoid -> multiChannelCNN_tanh: 0.2514 ± 0.0005
baselineCNN_tanh -> multiChannelCNN_relu: 0.2155 ± 0.0099
baselineCNN_tanh -> multiChannelCNN_sigmoid: 0.2226 ± 0.0843
baselineCNN_tanh -> multiChannelCNN_tanh: 0.0000 ± 0.0000


In [3]:
import torch
import os

def load_model_weights(model_path):
    """Load model weights from a .pth file."""
    checkpoint = torch.load(model_path, map_location=torch.device("cpu"))
    model_state = checkpoint["student_model_state_dict"]
    return {k: v.cpu().numpy() for k, v in model_state.items() if "weight" in k}

def compute_sparsity(weights_dict):
    """Compute the sparsity of model weights (per layer)."""
    sparsity = {}
    for layer_name, weights in weights_dict.items():
        num_elements = weights.size
        num_zeros = (abs(weights) < 1e-4).sum()
        sparsity[layer_name] = num_zeros / num_elements    
    return sparsity

def compute_layerwise_sparsity(directory):
    """Compute and print layer-wise sparsity for each model in a folder."""
    for root, _, files in os.walk(directory):
        for file in files:
            if not file.endswith(".pth"):
                continue

            model_path = os.path.join(root, file)
            weights_dict = load_model_weights(model_path)
            sparsity = compute_sparsity(weights_dict)

            print(f"\nModel: {file}")
            for layer_name, sparsity_value in sparsity.items():
                print(f"  {layer_name}: {sparsity_value * 100:.2f}% sparsity")

In [5]:
compute_layerwise_sparsity("../experiment_output/Experiment2/experiments_exp2_l2_init1_1")


Model: nonoverlappingCNN_sigmoid__overlappingCNN_sigmoid.pth
  layers.0.weight: 0.00% sparsity
  layers.1.weight: 0.00% sparsity
  layers.2.weight: 0.00% sparsity

Model: nonoverlappingCNN_tanh__overlappingCNN_sigmoid.pth
  layers.0.weight: 0.00% sparsity
  layers.1.weight: 0.00% sparsity
  layers.2.weight: 0.00% sparsity

Model: nonoverlappingCNN_relu__overlappingCNN_relu.pth
  layers.0.weight: 8.33% sparsity
  layers.1.weight: 9.38% sparsity
  layers.2.weight: 0.00% sparsity

Model: nonoverlappingCNN_tanh__overlappingCNN_tanh.pth
  layers.0.weight: 0.00% sparsity
  layers.1.weight: 0.00% sparsity
  layers.2.weight: 0.00% sparsity

Model: nonoverlappingCNN_relu__overlappingCNN_sigmoid.pth
  layers.0.weight: 0.00% sparsity
  layers.1.weight: 0.00% sparsity
  layers.2.weight: 0.00% sparsity

Model: nonoverlappingCNN_sigmoid__overlappingCNN_relu.pth
  layers.0.weight: 0.00% sparsity
  layers.1.weight: 0.00% sparsity
  layers.2.weight: 0.00% sparsity

Model: nonoverlappingCNN_sigmoid__ov

In [10]:
import os
import re
from collections import defaultdict

def parse_final_losses_from_file(filepath):
    with open(filepath, 'r') as file:
        content = file.read()

    # Split into blocks using separator line
    blocks = content.split("=" * 80)

    model_losses = defaultdict(list)

    for block in blocks:
        model_match = re.search(r'([^\s]+)\s*->\s*([^\n]+)', block)
        loss_match = re.search(r'Final Loss:\s*([0-9.]+)', block)

        if model_match and loss_match:
            teacher_model = model_match.group(1).strip()
            student_model = model_match.group(2).strip()
            loss = float(loss_match.group(1))

            pair_key = f"{teacher_model} -> {student_model}"
            model_losses[pair_key].append(loss)
    return model_losses

def collect_all_losses(folder_path):
    all_losses = defaultdict(list)

    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                file_losses = parse_final_losses_from_file(file_path)
                for model_pair, losses in file_losses.items():
                    all_losses[model_pair].extend(losses)

    # Print results
    for model_pair, losses in all_losses.items():
        print(f"{model_pair}:")
        for i, loss in enumerate(losses):
            print(f"  Run {i+1}: {loss}")
    
    #return all_losses


In [11]:
collect_all_losses("../experiment_output/Experiment2/experiments_exp2_l2_init1_1")

nonoverlappingCNN_sigmoid -> overlappingCNN_sigmoid:
  Run 1: 0.0001
nonoverlappingCNN_sigmoid -> overlappingCNN_relu:
  Run 1: 0.0121
nonoverlappingCNN_sigmoid -> overlappingCNN_tanh:
  Run 1: 0.2521
nonoverlappingCNN_relu -> overlappingCNN_sigmoid:
  Run 1: 3.3666
nonoverlappingCNN_relu -> overlappingCNN_relu:
  Run 1: 0.0001
nonoverlappingCNN_relu -> overlappingCNN_tanh:
  Run 1: 3.6672
nonoverlappingCNN_tanh -> overlappingCNN_sigmoid:
  Run 1: 0.1919
nonoverlappingCNN_tanh -> overlappingCNN_relu:
  Run 1: 0.2112
nonoverlappingCNN_tanh -> overlappingCNN_tanh:
  Run 1: 0.0002
